In [28]:
!pip install pycuda

Standard imports.


In [0]:
import numpy as np
import matplotlib.pyplot as plt

from matplotlib import animation, rc
from IPython.display import HTML

PyCUDA imports

In [0]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

```iDivUp``` function: if ```b``` divides ```a```, then ```a/b``` is returned, otherwise the function returns the integer division between ```a``` and ```b``` summed to 1```.

In [0]:
###################
# iDivUp FUNCTION #
###################
def iDivUp(a, b):
    # Round a / b to nearest higher integer value
    a = np.int32(a)
    b = np.int32(b)
    return (a / b + 1) if (a % b != 0) else (a / b)

Kernel functions. 

```initialConditionKernel```: sets the initial condition

```matsunoFirstStep```      : sets the first step according to Matsuno's scheme

```updateKernel```          : solution update 

The first two kernels use the ```propagatingFunction``` ```__device__``` function which defines the initial condition.

In [0]:
mod = SourceModule("""
   #include <stdio.h>
  
   #define PI_f			3.14159265358979323846  
   
   __host__ __device__  float propagatingFunction(const float x) { return exp(-x * x / (2.f * (PI_f / 4.f) * (PI_f / 4.f))); };

   __global__ void initialConditionKernel(float * __restrict__ d_u, const float * __restrict__ d_t, const float * __restrict__ d_x, const float v, const int N) {

	    const int tid = threadIdx.x + blockIdx.x * blockDim.x;

	    if (tid >= N + 1) return;

	    d_u[tid]			= propagatingFunction(d_x[tid] - v * d_t[0]);     // --- Initial condition 
      
    }

   __global__ void matsunoFirstStep(float * __restrict__ d_u, const float * __restrict__ d_t, const float * __restrict__ d_x, const float v, const float alpha, const int N) {

      const int tid = threadIdx.x + blockIdx.x * blockDim.x;

	    if (tid > N + 1) return;

	    // --- Matsuno initial condition
	    if (tid == 0) d_u[N + 1] = propagatingFunction(d_x[tid] - v * d_t[1]);			// --- Enforcing boundary condition (left boundary)
	    else d_u[tid + N + 1] = d_u[tid] - 0.5 * alpha * (d_u[tid + 1] - d_u[tid - 1]); }

   __global__ void updateKernel(float * __restrict__ d_u, const float * __restrict__ d_t, const float * __restrict__ d_x, const float v, 
	                         const float alpha, const float Q, const int l, const int N) {

	    const int tid = threadIdx.x + blockIdx.x * blockDim.x;

      if (tid >= N + 1) return;

	    if ((tid > 0) && (tid < N))
		
		      d_u[tid + (l + 1) * (N + 1)] = d_u[tid + (l - 1) * (N + 1)] - alpha * (d_u[tid + 1 + l * (N + 1)] - d_u[tid - 1 + l * (N + 1)]);
	
	    else if (tid == 0) {

		     d_u[tid + (l + 1) * (N + 1)] = propagatingFunction(d_x[tid] - v * d_t[l + 1]); }
	
      else if (tid == N) {

         d_u[tid + (l + 1) * (N + 1)] = (1.f - Q) * d_u[tid - 1 + l * (N + 1)] + Q * d_u[tid + l * (N + 1)]; }}
         
  """)

Set references to the ```__global__``` functions.


In [0]:
initialConditionKernel = mod.get_function("initialConditionKernel")
matsunoFirstStep       = mod.get_function("matsunoFirstStep")
updateKernel           = mod.get_function("updateKernel")

Define the block size.

In [0]:
BLOCKSIZE = 256

Solution function. Rows span time, columns span space. We have ```M + 1``` rows: ```M``` iterations + ```1``` initial condition. We have ```N + 1``` columns: ```N``` discretization points + ```1``` boundary condition.

Leapfrog.

In [0]:
def leapfrog(d_x, d_t, v, alpha):

  # --- Allocate device memory space for solution
  d_u = cuda.mem_alloc((N + 1) * (M + 1) * 4)
  # --- Set memory to zero
  cuda.memset_d32(d_u, 0x00, (N + 1) * (M + 1))

  # --- Initial condition
  blockDim  = (BLOCKSIZE, 1, 1)
  gridDim   = (int(iDivUp(N + 1, BLOCKSIZE)), 1, 1)
  initialConditionKernel(d_u, d_t, d_x, np.float32(v), np.int32(N), block = blockDim, grid = gridDim)

  # --- First step
  matsunoFirstStep(d_u, d_t, d_x, np.float32(v), np.float32(alpha), np.int32(N), block = blockDim, grid = gridDim)

  Q = (1. - alpha) / (1. + alpha)
  for l in range(1, M):
    updateKernel(d_u, d_t, d_x, np.float32(v), np.float32(alpha), np.float32(Q), np.int32(l), np.int32(N), block = blockDim, grid = gridDim);    # --- Boundary condition

  return d_u

Parameters.

In [0]:
xmin  = 0                                    # --- Left boundary of the simulation domain
xmax  = 2. * np.pi                           # --- Right boundary of the simulation domain
t_0   = 0.                                   # --- Initial time
t_f   = 15.                                  # --- Final time
M     = 200                                  # --- Number of time steps
N     = 165                                  # --- Number of space mesh points
v     = 0.5                                  # --- Wave speed

Space-time discretization on the host.

In [0]:
x, dx = np.linspace(xmin, xmax, N + 1, dtype = np.float32, retstep = True)

t, dt = np.linspace(t_0, t_f, M + 1, dtype = np.float32, retstep = True)
            
alpha = v * dt / dx

Moving discretization to the device.

In [0]:
d_x = cuda.mem_alloc(x.nbytes)
d_t = cuda.mem_alloc(t.nbytes)

cuda.memcpy_htod(d_x, x)
cuda.memcpy_htod(d_t, t)

Compute solution.

In [0]:
d_u = leapfrog(d_x, d_t, v, alpha)

Copy the solution from device to host.

In [0]:
u = np.zeros((M + 1, N + 1), dtype = np.float32)
cuda.memcpy_dtoh(u, d_u)

Propagating wavefront. It serves as reference.

In [0]:
def propagatingFunction(x):
    return np.exp(-np.power(x, 2) / (2. * np.power(np.pi / 4., 2)))

Define reference solution.

In [0]:
X, T = np.meshgrid(x, t)           

uRef = propagatingFunction(X - v * T)

Animate.

In [43]:
fig, ax = plt.subplots()
plt.close()

ax.set_xlim(( xmin, xmax))
ax.set_ylim((    0, 1.1))

line1, = ax.plot([], [], lw = 2, color = "r", linestyle = 'dashed')
line2, = ax.plot([], [], lw = 2, color = "b")

def animate(i):
    global x
    global u, uRef
    yRef = uRef[i]
    y    = u[i]
    line1.set_data(x, yRef)
    line2.set_data(x, y)
    return (line1, line2)

anim = animation.FuncAnimation(fig, animate, frames = M + 1, interval = 20)
anim.save('leapfrog.mp4', fps = 30, extra_args=['-vcodec', 'libx264'])
rc('animation', html = 'jshtml') 
anim